# Spatial Analysis

CARTOframes provides the `Dataset(your_query)` method for performing analysis and returning the results as a pandas dataframe, and the `Layer` class for visualizing analysis as a map layer. Both methods run the quries against a **PostgreSQL** database with **PostGIS**. CARTO also provides more advanced spatial analysis through the crankshaft extension.

In this guide, we will analyze McDonald’s locations in US Census tracts using spatial analysis functionality in CARTO.

You can download directly the dataset from their sources:

- [NYC Census Tracts Data](https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2018&layergroup=Census+Tracts)
- [NYC McDonalds](https://data.cityofnewyork.us/Health/McDonald-s/kyws-ad2t)

Or follow the following steps to download them directly from the CARTOframes account.

In [6]:
from cartoframes.auth import Credentials
from cartoframes.viz import Map, Layer, Legend, Source
from cartoframes.data import Dataset

## Add here your CARTO credentials:
credentials = Credentials(base_url='https://your_user_name.carto.com', api_key='your_api_key')
cf_credentials = Credentials(base_url='https://cartoframes.carto.com', api_key='default_public')

In [ ]:
mcdonalds_nyc_data = Dataset('mcdonalds_nyc', credentials=cf_credentials)
mcdonalds_nyc_data_df = mcdonalds_nyc_data.download()

mcdonalds_nyc = Dataset(mcdonalds_nyc_data_df)
mcdonalds_nyc.upload(table_name='mcdonalds_nyc', if_exists='replace', credentials=credentials)

In [ ]:
nyc_census_tracts_data = Dataset('nyc_census_tracts', credentials=cf_credentials)
nyc_census_tracts_data_df = nyc_census_tracts_data.download()

nyc_census_tracts = Dataset(nyc_census_tracts_data_df)
nyc_census_tracts.upload(table_name='nyc_census_tracts', if_exists='replace', credentials=credentials)

### Example 1

Find the number of McDonald’s in each census tract in New York City.

In [9]:
mcdonalds_per_census_tracts = Dataset('''
    SELECT
      tracts.geom_refs AS FIPS_code,
      tracts.the_geom as the_geom,
      COUNT(mcd.*) AS num_mcdonalds
    FROM nyc_census_tracts As tracts, mcdonalds_nyc As mcd
    WHERE ST_Intersects(tracts.the_geom, mcd.the_geom)
    GROUP BY tracts.geom_refs, tracts.the_geom
    ORDER BY num_mcdonalds DESC
''', credentials=credentials)

In [10]:
# Show first five entries of results
# Including FIPS code (unique digital identifier for census tracts) and the number of McDonald's
# Sorted by the number of McDonald's in descending order
mcdonalds_per_census_tracts = mcdonalds_per_census_tracts.download()
mcdonalds_per_census_tracts.head()

,fips_code,the_geom,num_mcdonalds
0,36061010100,0106000020E61000000100000001030000000100000007...,4.0
1,36061008200,0106000020E61000000100000001030000000100000006...,2.0
2,36061011900,0106000020E61000000100000001030000000100000005...,2.0
3,36047119800,0106000020E6100000010000000103000000010000000D...,2.0
4,36061001502,0106000020E61000000100000001030000000100000014...,2.0


### Example 2

Build 100 meter buffer area for each McDonald’s by updating the geometry.

In [ ]:
# create a new table and save it as 'nyc_mcdonalds_buffer_100m'.
nyc_mcdonalds_buffer_100m = Dataset(
    '''
        SELECT name, id, address, city, zip,
               ST_Buffer(the_geom::geography, 100)::geometry AS the_geom
        FROM mcdonalds_nyc
    ''',
    credentials=credentials)

# Show the first entry.
# 'geomtery' is Polygon type now.
nyc_mcdonalds_buffer_100m.upload(table_name='nyc_mcdonalds_buffer_100m', if_exists='replace', credentials=credentials)
nyc_mcdonalds_buffer_100m_df = nyc_mcdonalds_buffer_100m.download()
nyc_mcdonalds_buffer_100m_df.iloc[0, :]

To show the results of this query on a map, we can do the following:

In [12]:
from cartoframes.viz import Map, Layer

Map([
    Layer('nyc_mcdonalds_buffer_100m', 'color: lightgray width: 20', credentials=credentials),
    Layer('mcdonalds_nyc', 'color: red width: 3', credentials=credentials),
  ],
  viewport={'zoom': 13.00, 'lat': 40.74, 'lng': -73.98}
)

### Example 3

Apply k-means (k=5) spatial clustering for all McDonald’s in NYC, and visualize different clusters by color. Note: for more complicated queries, it is best to create a temporary table from the query and then visualize it.

In [ ]:
k_means_dataset = Dataset('''
       SELECT
         row_number() OVER () AS cartodb_id,
         c.cluster_no,
         c.the_geom,
         ST_Transform(c.the_geom, 3857) AS the_geom_webmercator
       FROM
         ((SELECT *
           FROM cdb_crankshaft.cdb_kmeans(
               'SELECT the_geom, cartodb_id, longitude, latitude FROM mcdonalds_nyc', 5)
          ) AS a
           JOIN
         mcdonalds_nyc AS b
         ON a.cartodb_id = b.cartodb_id
         ) AS c
        ''',
        credentials=credentials)

k_means_dataset.upload(table_name='mcdonalds_clusters', if_exists='replace', credentials=credentials)

In [14]:
Map(Layer('mcdonalds_clusters', 'color: ramp($cluster_no, Prism)', credentials=credentials))